### Welcome to the Vaani Demo Notebook.
#### Please make sure you have all the libraries installed (refer to `requirements.txt` document for help).

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
MAX_LENGTH = 256
# CUDA suport test
tf.config.list_physical_devices('GPU')

[]

In [2]:
# importing word2vec model
import spacy
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [6]:
# initializing all background prep that goes into prediction (open this cell for autopsy)
# loading the model
demomodel = tf.keras.models.load_model('./models/bhagvadgita/BEST_max_pooled_autoencoder.h5')
demoencoder = tf.keras.Model(demomodel.input, demomodel.layers[5].output)
gitaDemo = pd.read_csv('../dataset/bhagvadgita_encoded.csv')
gitaDemo = gitaDemo.drop(['Unnamed: 0'], axis=1)
gitaDemo = gitaDemo.drop([546], axis=0)
gitaDemo['nlp'] = [nlp(i) for i in gitaDemo['English Translation']]

# function definitions
def tensorgoBrr(df: pd.DataFrame, column: str):
    """this function returns tensors for max pooled, mean pooled, and all present (padded up or down to 256 vectors)
    the return is a tuple with the order max, mean, and all tokens respectively."""
    maxtokens = []
    meantokens = []
    alltokens = []

    for i in df[column]:
        doc = nlp(i)
        tokenlist = np.array([token.vector.get() for token in doc])
        maxnormlist = np.array([float(token.vector_norm) for token in doc])
        if MAX_LENGTH - tokenlist.shape[0] > 0:
            constructarr = np.zeros((MAX_LENGTH - tokenlist.shape[0], 300))
            alltokens.append(np.append(tokenlist, constructarr, axis=(0)))
        else:
            alltokens.append(tokenlist[:MAX_LENGTH])

        maxtokens.append(np.array(tokenlist[maxnormlist.argmax()]))
        meantokens.append(np.array(tokenlist.mean(axis=(0))))

    tmaxtokens = tf.convert_to_tensor(maxtokens)
    tmeantokens = tf.convert_to_tensor(meantokens)
    talltokens = tf.convert_to_tensor(alltokens)

    return (tmaxtokens, tmeantokens, talltokens)

def find_closest_gita_verse(token):
    start = time.time()
    # print("Let me Think", end='')
    # time.sleep(2)
    # print('.',end='')
    encoding = demoencoder.predict(token, verbose=0)
    # time.sleep(0.4)
    vectorComp = gitaDemo.iloc[:, -6:-1].to_numpy()
    # print('.',end='')
    # time.sleep(0.5)
    # similarity rating function
    diff = np.array([np.dot(encoding, i)/(np.linalg.norm(encoding)*np.linalg.norm(i)) for i in vectorComp])
    seriesSlice = gitaDemo.iloc[diff.argmax(), 1:4]
    #print('.')
    # time.sleep(0.83)
    # return a nicely formatted string.
    exect = time.time() - start
    return f"AM4's Output: Chapter {seriesSlice[0]}, Verse {seriesSlice[1]} of the Bhagvadgita, which says:\n\n{seriesSlice[2]}\nTime Taken: {exect}"

def find_spacy_output(text):
    start = time.time()
    vector = nlp(text)
    simArr = np.array([vector.similarity(i) for i in gitaDemo['nlp']])
    seriesSlice = gitaDemo.iloc[simArr.argmax(), 1:4]
    exect = time.time() - start
    return f"Base Vector Comparison Output: Chapter {seriesSlice[0]}, Verse {seriesSlice[1]} of the Bhagvadgita, which says:\n\n{seriesSlice[2]}\nTime Taken: {exect}"


OSError: No file or directory found at /models/bhagvadgita/BEST_max_pooled_autoencoder.h5

Run all the cells, but rerun this one to experience the demo again.

In [4]:
# DEMO CELL
text = input('What Troubles you, friend?')
print(f'Text Input: {text}' + '\n---------------------------------')
token, _, _ = tensorgoBrr(pd.DataFrame([text], columns=(['text'])), 'text')
print(find_closest_gita_verse(token) + '\n---------------------------------')
print(find_spacy_output(text) + '\n---------------------------------')

Text Input: what is the meaning of life
---------------------------------


NameError: name 'tensorgoBrr' is not defined

Thank you for your time